In [206]:
import pandas as pd
import numpy as np
import re

# Danh sách các từ khóa liên quan đến thành phần vải (danh sách gốc)
element_key = ['cotton', 'staple', 'poly', 'elastane', 
               'lycra', 'floral', 'modal', 'bông', 'rayon', 'tencel', 'lyocell', 'bcicotton', 'nylon', 'freefit',
               'wool', 'shrink', 'negastat', 'linen', 'carbon']

# Danh sách mở rộng với các từ khóa bổ sung liên quan đến thành phần vải
element_key_update = ['viscose', 'organic', 'acrylic', 'cashmere', 'acetate', 'ecovero', 'elasthane', 'glass', 'recycle',
                      'recycle cotton', 'spa', 'non-gmo organic', 'cocona', 'combed cotton', 'cupro',
                      'thermolite', 'urethane', 'acetater', 'idigo', 'spx', 'elastomultiester', 't-400', 't400', 
                      'supimacotton', 'coolmax', 'metallic', 'hemp']

# Danh sách các trường hợp đặc biệt cần xử lý bổ sung (danh sách mở rộng)
special_case_key_update = ['ly', 'ep', 'tel', 'cup', 'pa']

# Danh sách các trường hợp đặc biệt có thể xuất hiện trong dữ liệu gốc
special_case_key = ['co', 'pl', 'pu', 'c:', 'p:', 'r:', 'sp', 'rp', 'li', 'ea', 'te', 'rc']

# Hợp nhất danh sách từ khóa chính với danh sách cập nhật
element_key_afupdate = element_key + element_key_update

# Hợp nhất danh sách các trường hợp đặc biệt với danh sách cập nhật
special_case_key_afupdate = special_case_key + special_case_key_update


In [207]:
# Đọc dữ liệu từ file CSV, không có tiêu đề nên đặt tên cột là 'Text'
df = pd.read_csv('SP vải_fiber_final_export.csv', header=None, names=['Text'])

# Hàm trích xuất giá trị phần trăm từ văn bản nhưng loại trừ các mẫu không mong muốn
def extract_percent_values(text):
    """
    Trích xuất các giá trị phần trăm từ văn bản đầu vào, đồng thời loại bỏ những mẫu không mong muốn
    như: '100% new', 'new 100%', 'mới 100%', '100% sample', 'sample 100%', 'item 100%', v.v.
    """
    if pd.isna(text):  # Xử lý trường hợp giá trị bị thiếu
        return []
    
    # Chuyển văn bản về chữ thường để xử lý đồng nhất
    text = text.lower()
    
    # Tạo một bản sao của văn bản để che các mẫu cần loại trừ
    masked_text = text
    
    # Định nghĩa danh sách các mẫu cần loại trừ, đảm bảo khớp với nhiều định dạng khác nhau
    exclude_patterns = [
        r'new\s*100%',           # Matches 'new 100%' with any spaces
        r'100%\s*new',           # Matches '100% new' with any spaces
        r'100%new',              # Matches '100%new'
        r'\bnew\s*100%\b[\s,.\)]?',     # Matches 'new 100%' followed by space, comma, period, or closing parenthesis
        r'\b100%\s*new\b[\s,.\)]?',     # Matches '100% new' with optional space, comma, period, or closing parenthesis
        r'\b100%new\b[\s,.\)]?',         # Matches '100%new' with optional punctuation
        r'100%\s*brand',         # Matches '100% brand'
        r'100%brand',            # Matches '100%brand'
        r'100%\s*hàng\s*mới',    # Matches '100% hàng mới'
        r'100%hàng\s*mới',       # Matches '100%hàng mới'
        r'hàng\s*mới\s*100%',    # Matches 'hàng mới 100%'
        r'hàng\s*mới100%',       # Matches 'hàng mới100%'
        r'mới\s*100%',           # Matches 'mới 100%' with any spaces
        r'mới100%',              # Matches 'mới100%'
        r'100%\s*mới',           # Matches '100% mới' with any spaces
        r'100%mới',              # Matches '100%mới'
        r'100%\s*sample',        # Matches '100% sample'
        r'100%sample',           # Matches '100%sample'
        r'sample\s*100%',        # Matches 'sample 100%'
        r'sample100%',           # Matches 'sample100%'
        r'item\s*100%',          # Matches 'item 100%' with any spaces
        r'item100%'              # Matches 'item100%'
        r'\(\s*\+\-\s*\d+%\)',   # Matches "(+-5%)" where 5 can be any number
        r'\+\-\s*\d+%',          # Matches "+-5%" without parentheses
        r'\(\s*\+/\-\s*\d+%\)',  # Matches "(+/-5%)" where 5 can be any number
        r'\+/\-\s*\d+%'          # Matches "+/-5%" without parentheses
        r'\.\s*100%\s*[#><?@&.*]' #Matches ". 100%.", ". 100% $"
        #r'100%\s*new[,]?', r'new\s*100%[,]?', r'100%new[,]?',  # Matches '100% new' and '100%new,'
        r'100%\s*new[,.\s]', r'new\s*100%[,.\s]', r'100%new[,.\s]',  # Matches '100% new' with punctuation
        r'100%\s*[\#&><?@!*]'  # <-- This removes '100%' if followed by special characters

    ]
    
    # Thêm mẫu loại trừ bổ sung vào danh sách
    exclude_patterns.append(r'\(\s*\+\-\s*\d+%\)') # Loại bỏ mẫu "(+-5%)"
    
    # Thay thế các mẫu bị loại trừ bằng chuỗi '_EXCLUDED_' để tránh bị bắt nhầm
    for pattern in exclude_patterns:
        masked_text = re.sub(pattern, '_EXCLUDED_', masked_text, flags=re.IGNORECASE)
    
    # Tìm tất cả các giá trị phần trăm còn lại trong văn bản đã được xử lý
    all_percentages = re.findall(r'\d+\.?\d*%', masked_text)
    
    return all_percentages

# Áp dụng hàm để trích xuất giá trị phần trăm đã lọc vào cột mới
df['Extracted_Percentages'] = df['Text'].apply(extract_percent_values)

# Đếm số lượng giá trị phần trăm được trích xuất thành công
df['Success_Catch'] = df['Extracted_Percentages'].apply(len)

In [208]:
# Hàm 1: Đếm số lần xuất hiện của từ khóa trong văn bản
def match_keywords(text, keyword):
    """
    Tìm kiếm số lần xuất hiện của một từ khóa trong văn bản bằng regex.
    """
    escaped_keyword = re.escape(keyword)  # Đảm bảo các ký tự đặc biệt không gây lỗi regex
    pattern = rf'(?<!\w|\d)[-_<>?&#()@!%]*\d*%*\s*{escaped_keyword}[-_<>?&#()@!%\w]*'  # Biểu thức chính quy tìm từ khóa
    matches = re.findall(pattern, text, re.IGNORECASE)  # Tìm tất cả các kết quả khớp
    return len(matches)  # Trả về số lần xuất hiện của từ khóa

# Hàm 2: Cập nhật DataFrame với thông tin {số lần xuất hiện: [vị trí]}
def update_column_with_positions(row, key_list):
    """
    Thay thế số lần xuất hiện bằng dictionary {count: [vị trí]} nếu count > 0.
    """
    text = str(row['Text']).lower()  # Chuyển văn bản về chữ thường để xử lý

    for col in key_list:
        if row[col] > 0:  # Chỉ xử lý nếu từ khóa xuất hiện ít nhất một lần
            escaped_col = re.escape(col)  # Đảm bảo từ khóa không làm hỏng regex
            pattern = rf'(?<!\w|\d)[-_<>?&#()@!%]*\d*%*\s*{escaped_col}[-_<>?&#()@!%\w]*'  # Biểu thức tìm vị trí xuất hiện
            matches = [m.start() for m in re.finditer(pattern, text, re.IGNORECASE)]  # Lấy danh sách vị trí xuất hiện

            if matches:
                row[col] = {row[col]: matches}  # Thay giá trị bằng dictionary {count: [positions]}
    
    return row


# Bước 1: Tạo một ma trận chứa số 0 với cùng chỉ mục như DataFrame gốc
zero_matrix = pd.DataFrame(0, index=df.index.copy(), columns=element_key_afupdate)

# Chuyển nội dung cột 'Text' về chữ thường để tránh phân biệt hoa/thường
df['Text'] = df['Text'].str.lower()

# Bước 2: Đếm số lần xuất hiện của từng từ khóa và lưu vào ma trận zero_matrix
for index, row in df.iterrows():
    text = row['Text']
    for keyword in element_key_afupdate:
        occurrences = match_keywords(text, keyword)  # Đếm số lần xuất hiện của từ khóa
        if occurrences > 0:
            zero_matrix.at[index, keyword] = occurrences  # Ghi số lần xuất hiện vào ma trận

# Bước 3: Hợp nhất ma trận zero_matrix vào DataFrame gốc
df = pd.concat([df, zero_matrix], axis=1)

# Bước 4: Chuyển đổi số lần xuất hiện thành {count: [positions]}
df = df.apply(lambda row: update_column_with_positions(row, element_key_afupdate), axis=1)

# Hiển thị 5 dòng đầu tiên của DataFrame
df.head()

,Text,Extracted_Percentages,Success_Catch,cotton,staple,poly,elastane,lycra,floral,modal,...,acetater,idigo,spx,elastomultiester,t-400,t400,supimacotton,coolmax,metallic,hemp
0,text,[],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,{1: [75]},0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,"{2: [4, 76]}",0,0,{1: [88]},0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,{1: [74]},0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,{1: [29]},0,{1: [40]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
def get_position_matching(row):
    """
    Tìm vị trí của các giá trị phần trăm đã trích xuất trong văn bản, đảm bảo khớp chính xác.
    Xử lý tất cả các lần xuất hiện, bao gồm cả trùng lặp, đồng thời đảm bảo các vị trí được nhận diện đúng.
    """
    extracted_percentages = row['Extracted_Percentages']  # Lấy danh sách phần trăm đã trích xuất

    if not extracted_percentages:  # Nếu không có giá trị nào, trả về danh sách rỗng
        return []

    text = str(row['Text']).lower()  # Chuyển văn bản về chữ thường để tìm kiếm
    matches = []
    used_positions = set()  # Tạo tập hợp để theo dõi vị trí đã sử dụng, tránh trùng lặp sai

    for percent in extracted_percentages:
        # Regex nâng cao để tìm chính xác giá trị phần trăm, tránh bị dính vào số khác
        pattern = rf'(?<!\d){re.escape(percent)}(?!\d)'  # Đảm bảo phần trăm đứng độc lập
        match_positions = [m.start() for m in re.finditer(pattern, text)]  # Tìm tất cả vị trí xuất hiện

        # Xử lý các giá trị trùng lặp bằng cách đảm bảo số lần xuất hiện đúng như mong đợi
        count_needed = extracted_percentages.count(percent)  # Xác định số lần cần tìm

        # Chỉ chọn những vị trí chưa được sử dụng để tránh nhầm lẫn
        filtered_positions = [pos for pos in match_positions if pos not in used_positions]
        selected_positions = filtered_positions[:count_needed]  # Giữ đúng số lần cần tìm
        matches.extend(selected_positions)  # Thêm các vị trí hợp lệ vào danh sách kết quả
        used_positions.update(selected_positions)  # Đánh dấu các vị trí đã sử dụng

    return sorted(matches)  # Trả về danh sách vị trí theo thứ tự tăng dần

# Áp dụng hàm để tìm vị trí của các giá trị phần trăm trong từng dòng dữ liệu
df['position_matching'] = df.apply(get_position_matching, axis=1)

# Di chuyển cột 'position_matching' vào vị trí thứ 4 trong DataFrame
column_order = df.columns.tolist()
column_order.insert(3, column_order.pop(column_order.index('position_matching')))  # Chèn vào vị trí thứ 4

# Sắp xếp lại thứ tự các cột trong DataFrame
df = df[column_order]


In [210]:
def calculate_total(row, keys):
    """
    Tính tổng giá trị của các cột được chỉ định trong danh sách `keys`.
    - Nếu giá trị trong ô là một dictionary (dạng {count: [positions]}), lấy key (số lần xuất hiện) để cộng vào tổng.
    - Nếu giá trị bằng 0, cộng thêm 0 vào tổng (để đảm bảo không bỏ sót).
    """
    total = 0
    for col in keys:  # Lặp qua từng cột trong danh sách
        value = row[col]
        if isinstance(value, dict):  
            total += list(value.keys())[0]  # Trích xuất key (số lần xuất hiện) từ dictionary và cộng vào tổng
        elif value == 0:  
            total += 0  # Nếu giá trị là 0 thì cộng 0 vào (đảm bảo tính tổng đúng)
    return total

# Áp dụng hàm để tính tổng giá trị cho tất cả các dòng trong DataFrame
df["Total_Sum"] = df.apply(lambda row: calculate_total(row, element_key_afupdate), axis=1)

# Lấy danh sách các cột hiện có trong DataFrame
cols = df.columns.tolist()

# Bỏ cột 'Total_Sum' tạm thời khỏi danh sách để sắp xếp lại vị trí
cols.remove('Total_Sum')

# Xác định vị trí của cột 'Success_Catch' trong danh sách
success_catch_index = cols.index('Success_Catch')

# Chèn 'Total_Sum' ngay sau 'Success_Catch'
cols.insert(success_catch_index + 1, 'Total_Sum')

# Sắp xếp lại DataFrame theo danh sách cột mới
df = df[cols]

# Hiển thị 5 dòng đầu của DataFrame để kiểm tra kết quả
df.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,acetater,idigo,spx,elastomultiester,t-400,t400,supimacotton,coolmax,metallic,hemp
0,text,[],0,0,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],{1: [75]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,3,"[76, 88]","{2: [4, 76]}",0,0,{1: [88]},0,...,0,0,0,0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],{1: [74]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,3,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
# Hàm 1: Đếm số lần xuất hiện của từ khóa đặc biệt trong văn bản bằng cách sử dụng nhiều mẫu regex
def match_special_keywords(text, keyword):
    """Tìm số lần xuất hiện của một từ khóa đặc biệt bằng nhiều mẫu regex khác nhau."""
    
    escaped_keyword = re.escape(keyword)  # Xử lý các ký tự đặc biệt trong từ khóa để tránh lỗi regex
    
    # Định nghĩa các mẫu regex để bắt nhiều cách viết khác nhau của từ khóa
    pattern = rf'(?<![\w/#])(?:\d+%\s*)?{escaped_keyword}[:%](?:\s*\d+%)?(?!\w|[/#])'  # Từ khóa có thể có dấu ':' hoặc '%'
    pattern2 = rf'(?<![\w/#]){escaped_keyword}\d+%(?!\w|[/#])'  # Từ khóa theo sau bởi số %
    pattern3 = rf'(?<![\w/#])\d+%{escaped_keyword}(?!\w|[/#])'  # Số % trước từ khóa
    pattern4 = rf'(?<![\w/#]){escaped_keyword}\s+\d+%(?!\w|[/#])'  # Từ khóa cách số % bởi khoảng trắng
    pattern5 = rf'(?<![\w/#])\d+%\s+{escaped_keyword}(?!\w|[/#])'  # Số % cách từ khóa bởi khoảng trắng
    pattern6 = rf'(?<![\w/#])(?:\d+%\s*)?{escaped_keyword}(?:\s*\d+%)?(?![\w/#])'  # Bắt tất cả các trường hợp có hoặc không có %

    # Kết hợp tất cả các mẫu regex lại thành một chuỗi duy nhất
    combined_pattern = f'({pattern})|({pattern2})|({pattern3})|({pattern4})|({pattern5})|({pattern6})'
    
    # Tìm tất cả các kết quả phù hợp trong văn bản
    matches = re.findall(combined_pattern, text, re.IGNORECASE)
    
    return len(matches)  # Trả về số lần xuất hiện của từ khóa


# Hàm 2: Cập nhật vị trí xuất hiện của từ khóa đặc biệt trong DataFrame
def update_special_column_with_positions(row, key_list):
    """Thay thế số lần xuất hiện của từ khóa đặc biệt bằng {count: [positions]} nếu số lần xuất hiện > 0."""
    
    text = str(row['Text']).lower()  # Chuyển văn bản về chữ thường để tránh phân biệt hoa thường

    for col in key_list:
        if row[col] > 0:  # Nếu từ khóa xuất hiện trong văn bản
            escaped_col = re.escape(col)  # Xử lý từ khóa để tránh lỗi regex
            
            # Áp dụng các mẫu regex giống như trong hàm `match_special_keywords`
            pattern = rf'(?<![\w/#])(?:\d+%\s*)?{escaped_col}[:%](?:\s*\d+%)?(?!\w|[/#])'
            pattern2 = rf'(?<![\w/#]){escaped_col}\d+%(?!\w|[/#])'
            pattern3 = rf'(?<![\w/#])\d+%{escaped_col}(?!\w|[/#])'
            pattern4 = rf'(?<![\w/#]){escaped_col}\s+\d+%(?!\w|[/#])'
            pattern5 = rf'(?<![\w/#])\d+%\s+{escaped_col}(?!\w|[/#])'
            pattern6 = rf'(?<![\w/#])(?:\d+%\s*)?{escaped_col}(?:\s*\d+%)?(?![\w/#])'

            combined_pattern = f'({pattern})|({pattern2})|({pattern3})|({pattern4})|({pattern5})|({pattern6})'
            
            # Tìm tất cả vị trí xuất hiện của từ khóa trong văn bản
            matches = [m.start() for m in re.finditer(combined_pattern, text, re.IGNORECASE)]

            if matches:
                row[col] = {row[col]: matches}  # Cập nhật giá trị của ô thành {count: [positions]}

    return row


# Bước 1: Lọc các dòng trong DataFrame mà `Success_Catch` lớn hơn `Total_Sum`
df_greater = df[df['Success_Catch'] > df['Total_Sum']].copy()

# Bước 2: Xóa cột 'Total_Sum' khỏi DataFrame (vì sẽ tính lại sau)
df_greater.drop(columns=['Total_Sum'], inplace=True)

# Bước 3: Tạo một ma trận zero (DataFrame chỉ chứa giá trị 0) có cùng chỉ mục với `df_greater`
zero_matrix_special = pd.DataFrame(0, index=df_greater.index.copy(), columns=special_case_key_afupdate)

# Bước 4: Đếm số lần xuất hiện của từ khóa đặc biệt và lưu vào ma trận zero
for index, row in df_greater.iterrows():
    text = row['Text']
    
    for keyword in special_case_key_afupdate:
        occurrences = match_special_keywords(text, keyword)  # Đếm số lần xuất hiện của từ khóa đặc biệt
        if occurrences > 0:
            zero_matrix_special.at[index, keyword] = occurrences  # Lưu kết quả vào ma trận

# Bước 5: Ghép ma trận `zero_matrix_special` vào `df_greater`
df_greater = pd.concat([df_greater, zero_matrix_special], axis=1)

# Bước 6: Áp dụng `update_special_column_with_positions()` để thay thế số lần xuất hiện bằng {count: [positions]}
df_greater = df_greater.apply(lambda row: update_special_column_with_positions(row, special_case_key_afupdate), axis=1)

# Hiển thị 5 dòng đầu của DataFrame sau khi xử lý
df_greater.head()


,Text,Extracted_Percentages,Success_Catch,position_matching,cotton,staple,poly,elastane,lycra,floral,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
36,janl011#&vải 98% lyocell 2% other fiber k:56/5...,"[98%, 2%]",2,"[13, 25]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539,"v252#&vải dệt thoi 98% cotton 2% other fiber,đ...","[98%, 2%]",2,"[19, 30]",{1: [19]},0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
556,"v251#&vải dệt thoi 99% cotton 1% other fiber,đ...","[99%, 1%]",2,"[19, 30]",{1: [19]},0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
557,v39#&vải dệt thoi từ bông có tỷ trọng từ 85% t...,"[85%, 1%]",2,"[41, 79]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
579,"nl521#&vải dệt thoi, 98% cotton 2% pu khổ 48/5...","[98%, 2%]",2,"[21, 32]",{1: [21]},0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
# Bước 1: Tạo một DataFrame mới (zero_matrix_special) chứa toàn số 0
# - Index giống với `df`
# - Các cột là danh sách `special_case_key_afupdate`
zero_matrix_special = pd.DataFrame(0, index=df.index, columns=special_case_key_afupdate)

# Bước 2: Xóa cột 'Total_Sum' khỏi DataFrame `df` (vì sẽ được tính lại sau)
df.drop(columns=['Total_Sum'], inplace=True)

# Bước 3: Ghép `df` với `zero_matrix_special` để thêm các cột mới vào DataFrame
df = pd.concat([df, zero_matrix_special], axis=1)

# Bước 4: Cập nhật `df` bằng các giá trị từ `df_greater`
# - `df_greater` chứa các dòng đã được xử lý đặc biệt (lọc theo điều kiện `Success_Catch > Total_Sum`)
# - Hàm `update()` sẽ chỉ cập nhật giá trị tại các ô có dữ liệu mới trong `df_greater`
df.update(df_greater)

# Bước 5: Hiển thị 5 dòng đầu tiên của DataFrame sau khi cập nhật
df.head()


,Text,Extracted_Percentages,Success_Catch,position_matching,cotton,staple,poly,elastane,lycra,floral,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
0,text,[],0,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,[66],{1: [75]},0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,"[76, 88]","{2: [4, 76]}",0,0,{1: [88]},0,0,...,0,0,0,0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,[65],{1: [74]},0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,"[29, 40]",{1: [29]},0,{1: [40]},0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
# Bước 1: Gộp hai danh sách `element_key_afupdate` và `special_case_key_afupdate` thành một danh sách `keys`
keys = element_key_afupdate + special_case_key_afupdate

# Bước 2: Áp dụng hàm `calculate_total()` để tính tổng giá trị từ các cột trong `keys` cho từng dòng
df["Total_Sum"] = df.apply(lambda row: calculate_total(row, keys), axis=1)

# Bước 3: Lấy danh sách tất cả các cột hiện có trong DataFrame
cols = df.columns.tolist()

# Bước 4: Loại bỏ tạm thời cột 'Total_Sum' khỏi danh sách cột (để sắp xếp lại vị trí sau)
cols.remove('Total_Sum')

# Bước 5: Tìm vị trí của cột 'Success_Catch' trong danh sách cột
success_catch_index = cols.index('Success_Catch')

# Bước 6: Chèn 'Total_Sum' ngay sau 'Success_Catch'
cols.insert(success_catch_index + 1, 'Total_Sum')

# Bước 7: Sắp xếp lại DataFrame theo danh sách cột mới đã chỉnh sửa
df = df[cols]

# Bước 8: Hiển thị 5 dòng đầu tiên của DataFrame sau khi cập nhật
df.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
0,text,[],0,0,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],{1: [75]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,3,"[76, 88]","{2: [4, 76]}",0,0,{1: [88]},0,...,0,0,0,0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],{1: [74]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,3,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
# Bước 1: Lọc các dòng có 'Success_Catch' bằng 0 và lưu vào `df_trash_can1`
df_trash_can1 = df[df['Success_Catch'] == 0]  

# Bước 2: Giữ lại các dòng có 'Success_Catch' lớn hơn 0 trong DataFrame chính
df = df[df['Success_Catch'] > 0]  

# Bước 3: Trích xuất 30 dòng đầu tiên của DataFrame sau khi lọc
df_extract = df.head(30)  

# Bước 4: Xuất 30 dòng đầu tiên ra file CSV có tên 'df_extraction.csv'
df_extract.to_csv('df_extraction.csv')


In [215]:
# Bước 1: Lọc ra các dòng có 'Success_Catch' nhỏ hơn 'Total_Sum'
df_less = df[df['Success_Catch'] < df['Total_Sum']]

def handle_less_case(row, element_key=keys):
    """
    Xử lý các trường hợp khi 'Success_Catch' < 'Total_Sum' bằng cách điều chỉnh dữ liệu.
    """

    pm = row['position_matching']  # Danh sách vị trí đã khớp
    ep = row['Extracted_Percentages']  # Danh sách phần trăm trích xuất được
    delta = row['Total_Sum'] - row['Success_Catch']  # Số lượng cần loại bỏ
    cell_store = {}  # Dictionary để lưu giá trị theo từng cột

    # Khởi tạo danh sách rỗng cho mỗi cột
    for col in element_key:
        if col not in cell_store:
            cell_store[col] = []  
    
        val = row[col]
        if isinstance(val, dict):
            extracted_val = list(val.items())[0][1]  # Lấy giá trị từ dict {count: [positions]}
            cell_store[col].append(extracted_val)  # Lưu danh sách vị trí

    # Tạo DataFrame từ dữ liệu đã trích xuất
    df_temp = pd.DataFrame([(k, v) for k, values in cell_store.items() for v in values], 
                           columns=["Element", "Position"])

    # Chuyển đổi cột "Position" thành danh sách riêng lẻ nếu có nhiều giá trị
    df_temp = df_temp.explode("Position", ignore_index=True)

    # Đếm số lần xuất hiện của từng vị trí
    value_counts = df_temp["Position"].value_counts()

    # Xử lý trùng lặp vị trí
    for val in df_temp["Position"].unique():
        if value_counts[val] > 1:  # Nếu có trùng lặp
            sub_df = df_temp[df_temp["Position"] == val]  # Lọc ra các dòng có cùng vị trí
            
            # Tìm vị trí có tên ngắn nhất
            idx_to_keep = sub_df.loc[sub_df["Element"].str.len().idxmin()].name  
            
            # Đặt tất cả các vị trí trùng còn lại thành 0
            df_temp.loc[(df_temp["Position"] == val) & (df_temp.index != idx_to_keep), "Position"] = 0

    # Tính khoảng cách từ mỗi vị trí đến vị trí khớp gần nhất
    df_temp["Distance"] = df_temp["Position"].apply(lambda x: min(abs(x - pos) for pos in pm))

    # Nếu 'Position' = 0 và 'Distance' = 0, đổi thành 1 để tránh lỗi
    df_temp.loc[(df_temp['Position'] == 0) & (df_temp['Distance'] == 0), 'Position'] = 1

    # Kiểm tra nếu các giá trị trong cột 'Distance' là duy nhất hoặc delta = 1
    if (len(set(df_temp["Distance"].nlargest(delta).values)) > 1) or (delta == 1):
        df_temp.loc[df_temp["Distance"].nlargest(delta).index, "Position"] = 0  # Đặt các giá trị lớn nhất thành 0

    # Nhóm dữ liệu theo cột "Element" và tạo danh sách các vị trí
    final_df = df_temp.groupby("Element")["Position"].apply(list).reset_index()

    # Thêm cột đếm số lượng
    final_df["Count"] = final_df["Position"].apply(len)

    # Chuyển đổi thành dictionary có dạng {element: {count: [positions]}}
    temp = {row["Element"]: {row["Count"]: row["Position"]} for _, row in final_df.iterrows()}

    # Cập nhật giá trị vào dòng hiện tại của DataFrame
    for col in element_key:
        if isinstance(row[col], dict) and col in temp:
            row[col] = temp[col]

    return row

# Áp dụng xử lý cho từng dòng trong df_less
#df_less = df_less.apply(handle_less_case, axis=1)
df_less = df_less.apply(lambda row: handle_less_case(row, keys), axis=1)


# Hiển thị 5 dòng đầu của df_less
df_less.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,3,"[76, 88]","{2: [0, 76]}",0,0,{1: [88]},0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,3,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0,0,0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,3,"[17, 29]",{1: [17]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,vải dệt thoi từ sợi bông (100% cotton) đã tẩy ...,[100%],1,2,[26],{1: [25]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"wpcv18808#& vải dệt thoi từ bông, 100% cotton,...",[100%],1,2,[34],{1: [34]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
def clean_dict(value):
    """
    Loại bỏ các giá trị bằng 0 trong dictionary và cập nhật lại key tương ứng.
    
    - Nếu tất cả phần tử trong danh sách bị xóa (tức là toàn số 0) → Trả về 0.
    - Nếu danh sách còn lại sau khi xóa không rỗng → Cập nhật key sao cho không nhỏ hơn 1.
    """
    if isinstance(value, dict) and value:  # Kiểm tra xem giá trị có phải là dictionary và không rỗng
        key, lst = list(value.items())[0]  # Lấy cặp key-value đầu tiên từ dict
        
        # Lọc bỏ các phần tử có giá trị bằng 0 trong danh sách
        new_lst = [x for x in lst if x != 0]
        removed_zeros = len(lst) - len(new_lst)  # Đếm số lượng số 0 đã bị xóa
        
        if not new_lst:  # Nếu toàn bộ danh sách bị xóa (toàn số 0)
            return 0  
        
        # Giảm key xuống theo số lượng phần tử bị xóa, nhưng không nhỏ hơn 1
        new_key = max(1, key - removed_zeros)  
        
        return {new_key: new_lst}  # Trả về dictionary đã được làm sạch
    
    return value  # Nếu không phải dictionary, trả về giá trị ban đầu

def clean_row(row):
    """
    Áp dụng hàm clean_dict() cho tất cả các cột trong dataframe, 
    trừ cột 'Text'.
    """
    for col in keys:  # Duyệt qua danh sách các cột cần làm sạch
        row[col] = clean_dict(row[col])  # Cập nhật lại giá trị của cột
    return row

# Áp dụng hàm clean_row() lên từng dòng của df_less để làm sạch dữ liệu
df_less = df_less.apply(clean_row, axis=1)

# Hiển thị 5 dòng đầu của DataFrame sau khi xử lý
df_less.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,3,"[76, 88]",{1: [76]},0,0,{1: [88]},0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,3,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0,0,0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,3,"[17, 29]",{1: [17]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,vải dệt thoi từ sợi bông (100% cotton) đã tẩy ...,[100%],1,2,[26],{1: [25]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"wpcv18808#& vải dệt thoi từ bông, 100% cotton,...",[100%],1,2,[34],{1: [34]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [217]:
# Xóa cột 'Total_Sum' khỏi DataFrame df_less trước khi tính toán lại
df_less.drop(columns='Total_Sum', inplace=True)

# Tính lại tổng giá trị cho mỗi dòng dựa trên các cột trong `keys`
df_less["Total_Sum"] = df_less.apply(lambda row: calculate_total(row, keys), axis=1)

# Lấy danh sách tất cả các cột trong DataFrame
cols = df_less.columns.tolist()

# Loại bỏ 'Total_Sum' khỏi danh sách tạm thời để di chuyển vị trí của nó
cols.remove('Total_Sum')

# Tìm vị trí của cột 'Success_Catch' trong danh sách
success_catch_index = cols.index('Success_Catch')

# Chèn 'Total_Sum' ngay sau 'Success_Catch'
cols.insert(success_catch_index + 1, 'Total_Sum')

# Sắp xếp lại thứ tự cột trong DataFrame theo danh sách mới
df_less = df_less[cols]

# Hiển thị 5 dòng đầu tiên của DataFrame sau khi xử lý
df_less.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",{1: [76]},0,0,{1: [88]},0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0,0,0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",{1: [17]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,vải dệt thoi từ sợi bông (100% cotton) đã tẩy ...,[100%],1,1,[26],{1: [25]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"wpcv18808#& vải dệt thoi từ bông, 100% cotton,...",[100%],1,1,[34],{1: [34]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [218]:
# Cập nhật DataFrame df với các giá trị mới từ df_less
df.update(df_less)

# Hiển thị 5 dòng đầu tiên của DataFrame sau khi cập nhật
df.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],{1: [75]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",{1: [76]},0,0,{1: [88]},0,...,0,0.0,0,0.0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],{1: [74]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",{1: [29]},0,{1: [40]},0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",{1: [17]},0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0,0


In [219]:
# Hàm xử lý dữ liệu để cập nhật giá trị dựa trên vị trí và phần trăm trích xuất
def process_data(row, element_key=keys):
    ep = row['Extracted_Percentages']  # Lấy danh sách phần trăm đã trích xuất
    cell_store = {}  # Tạo một từ điển để lưu dữ liệu tạm thời

    # Xử lý các giá trị trong các cột chứa dictionary
    for col in element_key:
        val = row[col]
        if col not in cell_store:
            cell_store[col] = []  # Khởi tạo danh sách rỗng cho mỗi cột
        if isinstance(val, dict):
            extracted_val = list(val.values())[0]  # Lấy danh sách vị trí từ dictionary
            cell_store[col].extend(extracted_val)  # Lưu các giá trị vị trí vào danh sách

    # Chuyển dữ liệu từ dictionary sang DataFrame để xử lý dễ dàng hơn
    data = []
    for key, values in cell_store.items():
        for value in values:
            data.append([key, value])  # Lưu từng cặp [tên phần tử, vị trí]

    # Tạo DataFrame tạm thời từ danh sách dữ liệu
    df_temp = pd.DataFrame(data, columns=['Element', 'Position'])

    # Nếu cột 'Position' chứa danh sách, thì tách ra thành từng dòng riêng biệt
    df_temp = df_temp.explode('Position').reset_index(drop=True)

    # Chuyển đổi cột 'Position' sang kiểu số nguyên để sắp xếp
    df_temp['Position'] = df_temp['Position'].astype(int)

    # Sắp xếp DataFrame theo cột 'Position'
    df_temp = df_temp.sort_values(by='Position').reset_index(drop=True)

    # Gán giá trị 'Extracted_Percentages' theo thứ tự của 'Position'
    df_temp['Extracted_Percentages'] = ep[:len(df_temp)]  # Đảm bảo không bị lỗi độ dài

    # Tạo mapping từ vị trí gốc sang giá trị 'Extracted_Percentages' mới
    mapping = dict(zip(df_temp['Position'], df_temp['Extracted_Percentages']))

    # Cập nhật lại dictionary ban đầu dựa trên mapping mới
    transformed_dict = {key: [mapping[v] for v in values] for key, values in cell_store.items()}
    
    # Áp dụng thay đổi vào từng cột trong DataFrame
    for col in element_key:
        if isinstance(row[col], dict):
            if col in transformed_dict:  # Kiểm tra nếu cột có tồn tại trong dictionary mới
                row[col] = transformed_dict[col]  # Thay thế giá trị cũ bằng giá trị đã cập nhật
    return row


# Lọc các dòng có 'Success_Catch' bằng 'Total_Sum'
df_equal = df[df['Success_Catch'] == df['Total_Sum']]

# Áp dụng hàm process_data() cho từng dòng trong df_equal
df_equal = df_equal.apply(lambda row: process_data(row, keys), axis=1)

# Hiển thị DataFrame sau khi xử lý
df_equal


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",[97%],0,0,[3%],0,...,0,0.0,0,0.0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",[60%],0,[40%],0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",[70%],0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,"vn68-202308100045#&woven fabric, jacquard fabr...","[60%, 40%]",2,2,"[77, 88]",[60%],0,[40%],0,0,...,0,0,0,0,0,0,0,0,0,0
99997,"cotton woven fabric, 100% cotton of fabric 44""...",[100%],1,1,[21],[100%],0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
99998,"vn121-20230850131#&woven fabric, floral patter...","[97%, 3%]",2,2,"[78, 96]",[97%],0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
99999,"vn68-20230816003#&woven fabric, jacquard fabri...","[59%, 35%, 6%]",3,3,"[76, 87, 110]",[59%],0,0,[6%],0,...,0,0.0,0,0.0,0,0,0,0,0,0


In [220]:
# Import thư viện để xử lý chuỗi dưới dạng dictionary
import ast

# Hàm làm sạch giá trị trong ô nếu là dictionary
def clean_cell_value_corrected(value):
    """
    Làm sạch giá trị trong ô nếu là dictionary:
    - Xóa các giá trị bằng 0 trong danh sách.
    - Giữ nguyên các giá trị không phải danh sách nếu khác 0.
    - Nếu tất cả giá trị bị loại bỏ, trả về 0.
    """

    # Kiểm tra nếu giá trị là dictionary
    if isinstance(value, dict):
        cleaned_values = []  # Danh sách lưu các giá trị đã làm sạch

        for k, v in value.items():
            if isinstance(v, list):
                # Xóa các giá trị 0 khỏi danh sách
                non_zero_values = [i for i in v if i != 0]
                if non_zero_values:
                    cleaned_values.extend(non_zero_values)
            elif v != 0 and v != "0":  # Giữ giá trị nếu khác 0
                cleaned_values.append(v)

        # Trả về danh sách đã làm sạch hoặc 0 nếu danh sách trống
        return cleaned_values if cleaned_values else 0

    # Nếu không phải dictionary, giữ nguyên giá trị
    return value  

# Áp dụng hàm làm sạch cho toàn bộ DataFrame
df_equal = df_equal.applymap(clean_cell_value_corrected)

# Hiển thị DataFrame sau khi làm sạch
df_equal.head()


C:\Users\ACER NITRO\AppData\Local\Temp\ipykernel_9448\4011878155.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_equal = df_equal.applymap(clean_cell_value_corrected)


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",[97%],0,0,[3%],0,...,0,0.0,0,0.0,0,0,0,0,0,0
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",[60%],0,[40%],0,0,...,0,0.0,0,0.0,0,0,0,0,0,0
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",[70%],0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0,0


In [221]:
df_equal['Label']=0


In [222]:
# Hàm chuyển đổi dữ liệu thành dạng phần trăm có nhãn
def transform_to_percent(row, element_key=keys):
    """
    Biến đổi dữ liệu trong các cột của DataFrame thành chuỗi nhãn theo định dạng 'Tên Giá trị'.
    - Lưu trữ giá trị từ các cột được chỉ định vào dictionary `cell_store`.
    - Nếu giá trị là danh sách, duyệt qua từng phần tử và lưu vào `cell_store`.
    - Gộp tất cả các giá trị thành chuỗi nhãn và gán vào cột 'Label'.
    """

    cell_store = {}  # Dictionary lưu trữ các giá trị theo từng cột

    for col in element_key:
        val = row[col]

        if col not in cell_store:
            cell_store[col] = []  # Khởi tạo danh sách rỗng cho mỗi cột

        if isinstance(val, list):  # Nếu giá trị là danh sách
            for values in val:
                cell_store[col].append(values)  # Thêm từng giá trị vào danh sách

    # Gộp các giá trị thành chuỗi theo định dạng 'Tên Giá trị'
    row['Label'] = ', '.join([f"{key} {value}" for key, values in cell_store.items() for value in values])

    return row

# Áp dụng hàm chuyển đổi cho toàn bộ DataFrame
df_equal = df_equal.apply(lambda row: transform_to_percent(row, keys), axis=1)

# Hiển thị DataFrame sau khi xử lý
df_equal.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",[97%],0,0,[3%],0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",[60%],0,[40%],0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",[70%],0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 70%, carbon 30%"


In [223]:
df['Label']=0
df.update(df_equal)

In [224]:
df_selec = df[['Text','Label']]
df_selec.head()

,Text,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","cotton 70%, carbon 30%"


In [225]:
# Lọc các dòng có 'Success_Catch' khác 'Total_Sum' và chỉ lấy cột 'Text'
df_trash_can2 = df[df['Success_Catch'] != df['Total_Sum']]['Text']

# Chuyển đổi Series thành DataFrame để dễ thao tác
df_trash_can2 = pd.DataFrame(df_trash_can2)

# Hiển thị DataFrame sau khi xử lý
df_trash_can2.head()


,Text
36,janl011#&vải 98% lyocell 2% other fiber k:56/5...
539,"v252#&vải dệt thoi 98% cotton 2% other fiber,đ..."
556,"v251#&vải dệt thoi 99% cotton 1% other fiber,đ..."
557,v39#&vải dệt thoi từ bông có tỷ trọng từ 85% t...
589,"v297#&vải dệt thoi 97% cotton 3% others,đã nhu..."


In [226]:

def extract_percent_values(text):
    """
    Trích xuất các giá trị phần trăm từ văn bản, đồng thời loại bỏ một số mẫu cụ thể
    như '100% new', 'new 100%', '100% sample', v.v.
    """
    if pd.isna(text):  # Xử lý giá trị bị thiếu (NaN)
        return []

    text = text.lower()  # Chuyển văn bản thành chữ thường để chuẩn hóa dữ liệu
    
    # Các từ khóa không mong muốn đi kèm với %
    exclude_words = ['weight', 'by', 'value']
    # Tạo các mẫu regex để loại bỏ các từ khóa trên khi đi kèm với %
    additional_exclude_patterns = [rf'\b\d+%\s*{word}\b' for word in exclude_words]

    # Loại bỏ các mẫu từ danh sách trên khỏi văn bản
    for pattern in additional_exclude_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Các mẫu cần loại bỏ (chẳng hạn '100% new', '100% sample', v.v.)
    exclude_patterns = [
        r'\bnew\s*100%\b', r'\b100%\s*new\b', r'\b100%new\b',
        r'\b100%\s*brand\b', r'\b100%brand\b',
        r'\b100%\s*hàng\s*mới\b', r'\b100%hàng\s*mới\b',
        r'\bhàng\s*mới\s*100%\b', r'\bhàng\s*mới100%\b',
        r'\bmới\s*100%\b', r'\bmới100%\b', r'\b100%\s*mới\b', r'\b100%mới\b',
        r'\b100%\s*sample\b', r'\b100%sample\b', r'\bsample\s*100%\b', r'\bsample100%\b',
        r'\bitem\s*100%\b', r'\bitem100%\b',
        r'\(\s*\+\-\s*\d+%\)', r'\+\-\s*\d+%',  # Loại bỏ giá trị sai số "+- 5%"
        r'\(\s*\+/\-\s*\d+%\)', r'\+/\-\s*\d+%',  # Loại bỏ giá trị sai số "+/- 5%"
        r'\.\s*100%\s*[#><?@&.*]',  # Xóa '100%' khi đứng trước các ký tự đặc biệt
        r'\b100%\s*[\#&><?@!*]',  # Loại bỏ '100%' nếu theo sau là các ký tự đặc biệt
        r'\b100%\s*rating\b[:]*',  # Bỏ '100% rating' (hỗ trợ có hoặc không có dấu ":")
        r'\b100%\s*address\b[:]*'  # Bỏ '100% address' (hỗ trợ có hoặc không có dấu ":")
    ]

    # Loại bỏ các mẫu không mong muốn
    for pattern in exclude_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Tìm kiếm các thành phần phần trăm tiêu chuẩn có khoảng trắng (VD: "97% cotton")
    standard_compositions = re.findall(r'\b\d+\.?\d*%\s*\w+\b', text)

    # Tìm kiếm các thành phần viết liền nhau không có khoảng trắng (VD: "97%cotton3%spandex")
    continuous_compositions = re.findall(r'(\d+%[a-zA-Z]+)+', text)

    # Kết hợp cả hai loại thành phần
    all_compositions = standard_compositions + continuous_compositions

    return all_compositions


# Áp dụng hàm để trích xuất các giá trị phần trăm hợp lệ
df_trash_can2['Extracted_Percentages'] = df_trash_can2['Text'].apply(extract_percent_values)

# Đếm số lượng phần trăm trích xuất thành công trên mỗi dòng
df_trash_can2['Success_Catch'] = df_trash_can2['Extracted_Percentages'].apply(len)

# Hiển thị kết quả
df_trash_can2.head()


,Text,Extracted_Percentages,Success_Catch
36,janl011#&vải 98% lyocell 2% other fiber k:56/5...,"[98% lyocell, 2% other]",2
539,"v252#&vải dệt thoi 98% cotton 2% other fiber,đ...","[98% cotton, 2% other]",2
556,"v251#&vải dệt thoi 99% cotton 1% other fiber,đ...","[99% cotton, 1% other]",2
557,v39#&vải dệt thoi từ bông có tỷ trọng từ 85% t...,"[85% trở, 1% co]",2
589,"v297#&vải dệt thoi 97% cotton 3% others,đã nhu...","[97% cotton, 3% others]",2


In [227]:
# Giữ lại chỉ các cột cần thiết
df_trash_can2 = df_trash_can2[['Text', 'Extracted_Percentages']]

def remove_duplicates(fiber_list):
    """
    Loại bỏ các giá trị trùng lặp trong danh sách thành phần sợi.
    Nếu đầu vào không phải là danh sách, giữ nguyên giá trị.
    """
    return list(set(fiber_list)) if isinstance(fiber_list, list) else fiber_list

# Áp dụng hàm để loại bỏ các giá trị trùng lặp
df_trash_can2['Extracted_Percentages'] = df_trash_can2['Extracted_Percentages'].apply(remove_duplicates)

# Hiển thị kết quả
df_trash_can2.head()


,Text,Extracted_Percentages
36,janl011#&vải 98% lyocell 2% other fiber k:56/5...,"[98% lyocell, 2% other]"
539,"v252#&vải dệt thoi 98% cotton 2% other fiber,đ...","[98% cotton, 2% other]"
556,"v251#&vải dệt thoi 99% cotton 1% other fiber,đ...","[1% other, 99% cotton]"
557,v39#&vải dệt thoi từ bông có tỷ trọng từ 85% t...,"[85% trở, 1% co]"
589,"v297#&vải dệt thoi 97% cotton 3% others,đã nhu...","[3% others, 97% cotton]"


In [228]:
# Định nghĩa danh sách thay thế các từ cụ thể
replacement_dict = {
    'bci': 'cotton',
    'supima': 'cotton',
    'uncombed': 'uncombed cotton',
    'or': 'cotton',
    'carded': 'cotton',
    '40d': 'creona',
    'regenerativecotton': 'cotton',
    'cottob': 'cotton',
    'virgin': 'cotton',
    'trở': 'cotton'
}

# Hàm thay thế từ trong danh sách mà không làm thay đổi các phần khác
def replace_words_in_list(lst):
    """
    Kiểm tra nếu đầu vào là danh sách, thay thế các từ trong replacement_dict.
    Các từ không nằm trong danh sách sẽ được giữ nguyên.
    """
    if isinstance(lst, list):  # Đảm bảo chỉ xử lý khi đầu vào là danh sách
        return [
            re.sub(r'\b(' + '|'.join(replacement_dict.keys()) + r')\b', 
                   lambda match: replacement_dict[match.group(0)], word) 
            for word in lst
        ]
    return lst  # Giữ nguyên nếu không phải danh sách

# Áp dụng hàm thay thế cho cột 'Extracted_Percentages'
df_trash_can2['Extracted_Percentages'] = df_trash_can2['Extracted_Percentages'].apply(replace_words_in_list)

# Hiển thị DataFrame đã cập nhật
df_trash_can2.head()


,Text,Extracted_Percentages
36,janl011#&vải 98% lyocell 2% other fiber k:56/5...,"[98% lyocell, 2% other]"
539,"v252#&vải dệt thoi 98% cotton 2% other fiber,đ...","[98% cotton, 2% other]"
556,"v251#&vải dệt thoi 99% cotton 1% other fiber,đ...","[1% other, 99% cotton]"
557,v39#&vải dệt thoi từ bông có tỷ trọng từ 85% t...,"[85% cotton, 1% co]"
589,"v297#&vải dệt thoi 97% cotton 3% others,đã nhu...","[3% others, 97% cotton]"


In [229]:
df_trash_can2.to_csv("df_trash_eyes1.csv")

In [230]:
df_equal_label=df_equal[['Text','Label']]
df_equal_label.head()

,Text,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","cotton 70%, carbon 30%"


In [231]:
# result_demo=df_equal_label.to_csv('result_demo.csv')

# Xử lý trash_can1

In [232]:
def catch_and_positions(text):
    if isinstance(text, str):
        # Chỉ bắt số phần trăm (không liên quan đến vật liệu)
        pattern = r'\b(\d{1,3})\s*(pct|percent)\b'

        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        
        if matches:
            positions = [match.start() for match in matches]  # Vị trí xuất hiện
            found_percentages = [f"{match.group(1)}%" for match in matches]  # Chuyển về dạng "60%" hoặc "40%"
            return len(matches), positions, found_percentages
        
    return 0, [], ['No Label']


df_trash_can1['Success_Catch'], df_trash_can1['position_matching'], df_trash_can1['Extracted_Percentages'] = zip(*df_trash_can1.apply(
    lambda row: catch_and_positions(' '.join(row.astype(str))), axis=1))

cols = ['Text', 'Success_Catch', 'position_matching', 'Extracted_Percentages'] + [col for col in df_trash_can1.columns if col not in ['Text', 'Success_Catch', 'position_matching', 'Extracted_Percentages']]
df_trash_can1 = df_trash_can1[cols]

df_trash_can1.head()

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
0,text,0,[],[No Label],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313,p115#&vải dệt thoi từ bông đã nhuộm dán giấy (...,0,[],[No Label],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
386,"vải dệt thoi đã nhuộm, woven fabric 100 % bci ...",0,[],[No Label],1,{1: [46]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
387,"vải dệt thoi đã nhuộm, woven fabric 100 % bci ...",0,[],[No Label],1,{1: [46]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,"vải dệt thoi đã nhuộm, woven fabric 100 % bci ...",0,[],[No Label],1,{1: [46]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
df_valid_trash = df_trash_can1[df_trash_can1['position_matching'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
df_unvalid=df_trash_can1[df_trash_can1['position_matching'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
df_less_trash = df_valid_trash[df_valid_trash['Success_Catch'] < df_valid_trash['Total_Sum']]
print(df_less_trash.head())
df_less_trash

                                                    Text  Success_Catch  \
78761  vcvc01#&dyeed finished fabric 60 cotton 40 pol...              2   
84709  23f-v0190l3elv#&woven fabric from cotton v0190...              1   

      position_matching Extracted_Percentages  Total_Sum         cotton  \
78761          [63, 76]            [60%, 40%]          4  {2: [30, 69]}   
84709              [66]                [100%]          2  {2: [34, 74]}   

      staple           poly elastane lycra  ... rp li ea te rc ly ep tel cup  \
78761      0  {2: [40, 82]}        0     0  ...  0  0  0  0  0  0  0   0   0   
84709      0              0        0     0  ...  0  0  0  0  0  0  0   0   0   

      pa  
78761  0  
84709  0  

[2 rows x 68 columns]


,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
78761,vcvc01#&dyeed finished fabric 60 cotton 40 pol...,2,"[63, 76]","[60%, 40%]",4,"{2: [30, 69]}",0,"{2: [40, 82]}",0,0,...,0,0,0,0,0,0,0,0,0,0
84709,23f-v0190l3elv#&woven fabric from cotton v0190...,1,[66],[100%],2,"{2: [34, 74]}",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
df_less_trash = df_less_trash.apply(lambda row: handle_less_case(row, keys), axis=1)

df_less_trash.head()

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
78761,vcvc01#&dyeed finished fabric 60 cotton 40 pol...,2,"[63, 76]","[60%, 40%]",4,"{2: [0, 69]}",0,"{2: [0, 82]}",0,0,...,0,0,0,0,0,0,0,0,0,0
84709,23f-v0190l3elv#&woven fabric from cotton v0190...,1,[66],[100%],2,"{2: [0, 74]}",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [235]:
# Apply transformation row-wise
df_less_trash=df_less_trash.apply(clean_row,axis=1)
df_less_trash.head()

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
78761,vcvc01#&dyeed finished fabric 60 cotton 40 pol...,2,"[63, 76]","[60%, 40%]",4,{1: [69]},0,{1: [82]},0,0,...,0,0,0,0,0,0,0,0,0,0
84709,23f-v0190l3elv#&woven fabric from cotton v0190...,1,[66],[100%],2,{1: [74]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
df_less_trash.drop(columns='Total_Sum',inplace=True)
df_less_trash["Total_Sum"] = df_less_trash.apply(lambda row: calculate_total(row, keys), axis=1)

cols = df_less_trash.columns.tolist()

# Bỏ cột 'Total_Sum' tạm thời khỏi danh sách
cols.remove('Total_Sum')

# Tìm vị trí của cột 'Success_Catch'
success_catch_index = cols.index('Success_Catch')

# Chèn 'Total_Sum' ngay sau 'Success_Catch'
cols.insert(success_catch_index + 1, 'Total_Sum')

# Sắp xếp lại DataFrame theo danh sách cột mới
df_less_trash = df_less_trash[cols]
df_less_trash.head()

,Text,Success_Catch,Total_Sum,position_matching,Extracted_Percentages,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
78761,vcvc01#&dyeed finished fabric 60 cotton 40 pol...,2,2,"[63, 76]","[60%, 40%]",{1: [69]},0,{1: [82]},0,0,...,0,0,0,0,0,0,0,0,0,0
84709,23f-v0190l3elv#&woven fabric from cotton v0190...,1,1,[66],[100%],{1: [74]},0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [237]:
df_valid_trash.update(df_less_trash)
df_valid_trash.head()

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
651,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4361,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4581,23f-v0060f1e1v#&vải dệt thoi từ bông v0060f1e1...,1,[51],[100%],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26521,sm22-1021-202209-0053asl#&vải dệt kim vs-2040v...,2,"[48, 64]","[95%, 5%]",2,0,0,{1: [54]},0,0,...,0,0,0,0,0,0,0,0,0,0
31093,sm22-1011-202208-0525sip#&vải dệt kim vs-2040v...,2,"[58, 73]","[95%, 05%]",2,0,0,{1: [47]},0,0,...,0,0,0,0,0,0,0,0,0,0


In [238]:
df_greater_trash = df_valid_trash[df_valid_trash['Success_Catch'] > df_valid_trash['Total_Sum']]
df_greater_trash['Label'] = 'No Label'
df_selec_greater_trash = df_greater_trash[['Text','Label']]
df_selec_greater_trash.head()

,Text,Label


In [239]:
df_selec_greater_trash.to_csv("df_trash_eyes2.csv")

In [240]:
df_equal_trash = df_valid_trash[df_valid_trash['Success_Catch'] == df_valid_trash['Total_Sum']]
# Function to clean up dictionary-like cell values according to the specified rules

df_equal_trash = df_equal_trash.apply(lambda row: process_data(row, special_case_key_afupdate), axis=1)

# Apply the improved cleaning function to all cells in the DataFrame
df_equal_trash = df_equal_trash.applymap(clean_cell_value_corrected)
df_equal_trash.head()

df_equal_trash.head()

C:\Users\ACER NITRO\AppData\Local\Temp\ipykernel_9448\2313674898.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_equal_trash = df_equal_trash.applymap(clean_cell_value_corrected)


,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,rp,li,ea,te,rc,ly,ep,tel,cup,pa
651,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4361,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4581,23f-v0060f1e1v#&vải dệt thoi từ bông v0060f1e1...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
26521,sm22-1021-202209-0053asl#&vải dệt kim vs-2040v...,2,"[48, 64]","[95%, 5%]",2,0,0.0,[54],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
31093,sm22-1011-202208-0525sip#&vải dệt kim vs-2040v...,2,"[58, 73]","[95%, 05%]",2,0,0.0,[47],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [241]:
df_equal_trash['Label']=0
df_equal_trash

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
651,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4361,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4581,23f-v0060f1e1v#&vải dệt thoi từ bông v0060f1e1...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
26521,sm22-1021-202209-0053asl#&vải dệt kim vs-2040v...,2,"[48, 64]","[95%, 5%]",2,0,0.0,[54],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
31093,sm22-1011-202208-0525sip#&vải dệt kim vs-2040v...,2,"[58, 73]","[95%, 05%]",2,0,0.0,[47],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
56244,end20-vs-2040#&vải dệt kim vs-2040/ polyester ...,2,"[46, 60]","[95%, 5%]",2,0,0.0,[35],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
56245,end20-vs-2044-1#&vải dệt kim vs-2044-1/ polyes...,2,"[50, 64]","[95%, 5%]",2,0,0.0,[39],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
56909,end20-vs-2040#&vải dệt kim vs-2040/ polyester ...,2,"[46, 60]","[95%, 5%]",2,0,0.0,[35],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
63763,vải dệt kim đã nhuộm thành phẩm: 100pct polyes...,1,[33],[100%],1,0,0.0,[40],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
74242,23f-v0010k1e7v#&cotton woven fabric v0010k1e7v...,1,[50],[100%],1,[15],0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [242]:
df_equal_trash = df_equal_trash.apply(lambda row: transform_to_percent(row, special_case_key_afupdate), axis=1)
df_equal_trash.head()

,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
651,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
4361,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
4581,23f-v0060f1e1v#&vải dệt thoi từ bông v0060f1e1...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
26521,sm22-1021-202209-0053asl#&vải dệt kim vs-2040v...,2,"[48, 64]","[95%, 5%]",2,0,0.0,[54],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
31093,sm22-1011-202208-0525sip#&vải dệt kim vs-2040v...,2,"[58, 73]","[95%, 05%]",2,0,0.0,[47],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,


In [243]:
df_valid_trash['Label']=0
df_valid_trash.update(df_equal_trash)
df_valid_trash

C:\Users\ACER NITRO\AppData\Local\Temp\ipykernel_9448\1557094702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_trash['Label']=0


,Text,Success_Catch,position_matching,Extracted_Percentages,Total_Sum,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
651,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
4361,23f-v0010k1e7v#&vải dệt thoi từ bông v0010k1e7...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
4581,23f-v0060f1e1v#&vải dệt thoi từ bông v0060f1e1...,1,[51],[100%],1,0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
26521,sm22-1021-202209-0053asl#&vải dệt kim vs-2040v...,2,"[48, 64]","[95%, 5%]",2,0,0.0,[54],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
31093,sm22-1011-202208-0525sip#&vải dệt kim vs-2040v...,2,"[58, 73]","[95%, 05%]",2,0,0.0,[47],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
56244,end20-vs-2040#&vải dệt kim vs-2040/ polyester ...,2,"[46, 60]","[95%, 5%]",2,0,0.0,[35],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
56245,end20-vs-2044-1#&vải dệt kim vs-2044-1/ polyes...,2,"[50, 64]","[95%, 5%]",2,0,0.0,[39],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
56909,end20-vs-2040#&vải dệt kim vs-2040/ polyester ...,2,"[46, 60]","[95%, 5%]",2,0,0.0,[35],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
63763,vải dệt kim đã nhuộm thành phẩm: 100pct polyes...,1,[33],[100%],1,0,0.0,[40],0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,
74242,23f-v0010k1e7v#&cotton woven fabric v0010k1e7v...,1,[50],[100%],1,[15],0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,


In [244]:
df_selected_valid_trash = df[['Text', 'Label']]
df_selected_valid_trash.head()

,Text,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","cotton 70%, carbon 30%"


In [245]:
df.update(df_valid_trash)
df.head()


,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",[97%],0,0,[3%],0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",[60%],0,[40%],0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",[70%],0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 70%, carbon 30%"


In [246]:
# Thay thế giá trị 0 trong Label thành "No Label"
df['Label'] = df['Label'].apply(lambda x: "No Label" if x == 0 else x)
# Hiển thị kết quả
df.head()

,Text,Extracted_Percentages,Success_Catch,Total_Sum,position_matching,cotton,staple,poly,elastane,lycra,...,li,ea,te,rc,ly,ep,tel,cup,pa,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",[100%],1,1,[66],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","[97%, 3%]",2,2,"[76, 88]",[97%],0,0,[3%],0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",[100%],1,1,[65],[100%],0,0,0,0,...,0,0,0,0,0,0,0,0,0,cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"[60%, 40%]",2,2,"[29, 40]",[60%],0,[40%],0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","[70%, 30%]",2,2,"[17, 29]",[70%],0,0,0,0,...,0.0,0,0.0,0,0,0,0,0,0,"cotton 70%, carbon 30%"


In [247]:
# Lọc chỉ lấy 2 cột Text và Label
df_selected = df[['Text', 'Label']]
# Thay thế "spa" thành "Spandex" trong cột Label
df_selected['Label'] = df_selected['Label'].replace('spa', 'spandex', regex=True)

# Hiển thị kết quả
df_selected.head()

C:\Users\ACER NITRO\AppData\Local\Temp\ipykernel_9448\774972275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Label'] = df_selected['Label'].replace('spa', 'spandex', regex=True)


,Text,Label
1,"vn121-202304060007#&vải dệt thoi, vải vân hoa ...",cotton 100%
2,"vải cotton dệt thoi 80a400379, đã nhuộm, vải d...","cotton 97%, elastane 3%"
3,"sp-010-2023#&vải dệt thoi vân điểm, từ các sợi...",cotton 100%
4,npl211#&vải dệt thoi từ bông 60% cotton 40% po...,"cotton 60%, poly 40%"
5,"vải denim fabric 70% cotton, 30% carbon zero t...","cotton 70%, carbon 30%"


In [248]:
df_selected.to_csv("df_final.csv")